# CMS Open Payments Datalake Setup

**Project:** AAI-540 Machine Learning Operations - Final Team Project  
**Purpose:** Setup AWS S3 Datalake for CMS Open Payments Data  
**Dataset:** CMS Open Payments Program Year 2024

---

## Table of Contents
1. [Environment Setup](#setup)
2. [AWS Configuration & S3 Bucket Creation](#aws-config)
3. [Download CMS Open Payments Data](#download)
4. [Upload Data to S3](#upload)
5. [Create Athena Database](#athena)
6. [Register Data with Athena](#register)
7. [Convert CSV to Parquet](#parquet)
8. [Query Data with AWS Data Wrangler](#query)
9. [Validation & Verification](#validation)

---

## 1. Environment Setup

Install and import necessary libraries for AWS integration and data processing.

In [1]:
# Install required AWS packages
%pip install boto3 sagemaker awswrangler pyathena

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import necessary libraries
import boto3
import sagemaker
import pandas as pd
import numpy as np
import os
import requests
from pathlib import Path
from datetime import datetime
from io import BytesIO, StringIO
import awswrangler as wr
from pyathena import connect
import warnings

warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

print("Libraries imported successfully")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
Libraries imported successfully


## 2. AWS Configuration & S3 Bucket Creation

Configure AWS session and create S3 bucket for the datalake.

In [3]:
import boto3

# Initialize AWS session
boto_session = boto3.Session()
region = boto_session.region_name

# Get account information
sts_client = boto3.client('sts')
account_id = sts_client.get_caller_identity().get('Account')

# Initialize AWS clients
s3_client = boto3.client('s3', region_name=region)
s3_resource = boto3.resource('s3')

# Dataset file name
dataset_file_name = "lightdataset.csv" #"OP_DTL_GNRL_PGYR2024_P06302025_06162025.csv"

# Define bucket name
bucket = f"cmsopenpaymentsystemslight"
cms_data_prefix = "cms-open-payments-light"

# Define Athena database name
database_name = "cms_open_payments_light"

# Get IAM role (if needed)
iam = boto3.client('iam')
try:
    role = iam.get_role(RoleName='LabRole')['Role']['Arn']
except:
    role = "Role not found"

print(f"AWS Configuration:")
print(f"  Region: {region}")
print(f"  Account ID: {account_id}")
print(f"  S3 Bucket: {bucket}")
print(f"  Role: {role}")

# Check if S3 bucket exists
def check_bucket_exists(bucket_name):
    """Check if bucket exists and is accessible"""
    try:
        s3_client.head_bucket(Bucket=bucket_name)
        return True
    except:
        return False

def ensure_bucket_exists(bucket_name, region):
    """Create bucket only if it doesn't exist"""
    try:
        if check_bucket_exists(bucket_name):
            print(f"Bucket already exists: {bucket_name}")
            return True
        
        # Create bucket if it doesn't exist
        if region == 'us-east-1':
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            s3_client.create_bucket(
                Bucket=bucket_name,
                CreateBucketConfiguration={'LocationConstraint': region}
            )
        print(f"Created new bucket: {bucket_name}")
        return True
    except s3_client.exceptions.BucketAlreadyOwnedByYou:
        print(f"Bucket already exists: {bucket_name}")
        return True
    except Exception as e:
        print(f"Error with bucket {bucket_name}: {str(e)}")
        return False

# Ensure bucket exists
bucket_exists = ensure_bucket_exists(bucket, region)


AWS Configuration:
  Region: us-east-1
  Account ID: 092122669768
  S3 Bucket: cmsopenpaymentsystemslight
  Role: arn:aws:iam::092122669768:role/LabRole
Bucket already exists: cmsopenpaymentsystemslight


In [4]:
# Define S3 paths for CMS data
raw_data_prefix = f"{cms_data_prefix}/raw"
processed_data_prefix = f"{cms_data_prefix}/processed"
parquet_data_prefix = f"{cms_data_prefix}/parquet"

s3_raw_path = f"s3://{bucket}/{raw_data_prefix}"
s3_processed_path = f"s3://{bucket}/{processed_data_prefix}"
s3_parquet_path = f"s3://{bucket}/{parquet_data_prefix}"

print(f"S3 Data Paths:")
print(f"  Raw Data: {s3_raw_path}")
print(f"  Processed Data: {s3_processed_path}")
print(f"  Parquet Data: {s3_parquet_path}")

# Check if data already exists in S3
def check_s3_data_exists(bucket_name, prefix):
    """Check if data exists in S3 path"""
    try:
        response = s3_client.list_objects_v2(
            Bucket=bucket_name,
            Prefix=prefix,
            MaxKeys=1
        )
        return 'Contents' in response and len(response['Contents']) > 0
    except Exception as e:
        print(f"Error checking S3 path {prefix}: {e}")
        return False

# Check for existing data
raw_data_exists = check_s3_data_exists(bucket, raw_data_prefix)
parquet_data_exists = check_s3_data_exists(bucket, parquet_data_prefix)

print(f"\nData Status:")
print(f"  Raw Data in S3: {'EXISTS' if raw_data_exists else 'NOT FOUND'}")
print(f"  Parquet Data in S3: {'EXISTS' if parquet_data_exists else 'NOT FOUND'}")

if raw_data_exists and parquet_data_exists:
    print(f"\nData already ingested. You can skip to Step 5 (Create Athena Database)")
    skip_ingestion = True
elif raw_data_exists:
    print(f"\nRaw data exists. You may skip to Step 7 (Convert to Parquet)")
    skip_ingestion = True
else:
    print(f"\nData ingestion required. Continue with Steps 3-4.")
    skip_ingestion = False

# Store paths and flags for use in other notebooks
%store bucket
%store region
%store s3_raw_path
%store s3_processed_path
%store s3_parquet_path
%store raw_data_exists
%store parquet_data_exists
%store skip_ingestion


S3 Data Paths:
  Raw Data: s3://cmsopenpaymentsystemslight/cms-open-payments-light/raw
  Processed Data: s3://cmsopenpaymentsystemslight/cms-open-payments-light/processed
  Parquet Data: s3://cmsopenpaymentsystemslight/cms-open-payments-light/parquet

Data Status:
  Raw Data in S3: EXISTS
  Parquet Data in S3: EXISTS

Data already ingested. You can skip to Step 5 (Create Athena Database)
Stored 'bucket' (str)
Stored 'region' (str)
Stored 's3_raw_path' (str)
Stored 's3_processed_path' (str)
Stored 's3_parquet_path' (str)
Stored 'raw_data_exists' (bool)
Stored 'parquet_data_exists' (bool)
Stored 'skip_ingestion' (bool)


## 3. Download CMS Open Payments Data

Download the CMS Open Payments Program Year 2024 General Payments dataset.

**Data Source:** CMS Open Payments  
**Dataset:** Program Year 2024 General Payments  
**Published:** June 30, 2025  
**Coverage:** January 1, 2024 - December 31, 2024

In [5]:
# CMS Open Payments data URL - Direct CSV download
cms_data_url = "https://download.cms.gov/openpayments/PGYR2024_P06302025_06162025/OP_DTL_GNRL_PGYR2024_P06302025_06162025.csv"

# Alternative: If the above URL doesn't work, use this approach:
# 1. Go to https://openpaymentsdata.cms.gov/datasets
# 2. Select "Program Year 2024" and "General Payments"
# 3. Download the CSV file manually and place it in ../data/ directory

if skip_ingestion and raw_data_exists:
    print(f"Skipping download - data already exists in S3")
    print(f"  S3 Path: {s3_raw_path}")
else:
    print(f"CMS Data URL: {cms_data_url}")
    print(f"\nNote: This dataset is approximately 3-4 GB.")
    print(f"Download may take several minutes depending on your connection.")

Skipping download - data already exists in S3
  S3 Path: s3://cmsopenpaymentsystemslight/cms-open-payments-light/raw


In [6]:
# Create local data directory if it doesn't exist
local_data_dir = Path("../data")
local_data_dir.mkdir(exist_ok=True)

if skip_ingestion and raw_data_exists:
    print(f"Skipping download - data already exists in S3")
    print(f"  S3 Path: {s3_raw_path}")
    local_csv_file = local_data_dir / "headers.csv"
    print(f"Headers file: {local_csv_file.name}")
else:
    # Local CSV file path
    local_csv_file = local_data_dir / dataset_file_name
    print(f"Local data directory: {local_data_dir.absolute()}")
    print(f"Target CSV file: {local_csv_file.name}")

Skipping download - data already exists in S3
  S3 Path: s3://cmsopenpaymentsystemslight/cms-open-payments-light/raw
Headers file: headers.csv


In [7]:
# Download CMS data if not already present
if skip_ingestion and raw_data_exists:
    print(f"Skipping download - data already exists in S3")
    print(f"  S3 Path: {s3_raw_path}")
    
    # Check local file for reference
    if local_csv_file.exists():
        print(f"  Local file also present: {local_csv_file}")
        print(f"  File size: {local_csv_file.stat().st_size / (1024**3):.2f} GB")
    else:
        print(f"  Note: Local file not present, but S3 data is available")

elif local_csv_file.exists():
    print(f"CSV file already exists locally: {local_csv_file}")
    print(f"  File size: {local_csv_file.stat().st_size / (1024**3):.2f} GB")
    print(f"  Skipping download")
else:
    print(f"Downloading CMS Open Payments data...")
    print(f"  This may take 10-20 minutes depending on your connection.")
    
    try:
        # Download CSV file with progress indication
        response = requests.get(cms_data_url, stream=True)
        response.raise_for_status()
        
        total_size = int(response.headers.get('content-length', 0))
        print(f"  Total download size: {total_size / (1024**3):.2f} GB")
        
        # Save CSV file directly
        with open(local_csv_file, 'wb') as f:
            downloaded = 0
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
                    downloaded += len(chunk)
                    if total_size > 0:
                        percent = (downloaded / total_size) * 100
                        print(f"\rProgress: {percent:.1f}%", end="")
        
        print(f"\nDownload complete: {local_csv_file}")
        print(f"  File size: {local_csv_file.stat().st_size / (1024**3):.2f} GB")
            
    except Exception as e:
        print(f"\nError downloading data: {e}")
        print(f"\nAlternative approach:")
        print(f"1. Visit: https://openpaymentsdata.cms.gov/datasets")
        print(f"2. Select 'Program Year 2024' and 'General Payments'")
        print(f"3. Download CSV and save to: {local_data_dir.absolute()}")


Skipping download - data already exists in S3
  S3 Path: s3://cmsopenpaymentsystemslight/cms-open-payments-light/raw
  Local file also present: ../data/headers.csv
  File size: 0.00 GB


## 4. Upload Data to S3 upload

Upload the downloaded CMS data to S3 for datalake storage.

In [8]:
# Preview the data before upload
if skip_ingestion and raw_data_exists:
    print("Skipping preview - data already in S3")
    print(f"  Run queries in Step 6 to view data")
elif local_csv_file.exists():
    print("Loading sample of data for preview...")
    df_sample = pd.read_csv(local_csv_file, nrows=5)

    print(f"\nDataset Preview:")
    print(f"  Columns: {len(df_sample.columns)}")
    print(f"  Sample rows:")
    display(df_sample.head())

    print(f"\nColumn names:")
    for i, col in enumerate(df_sample.columns, 1):
        print(f"  {i}. {col}")
else:
    print("Local CSV file not found. Cannot preview data.")
    print(f"  Expected location: {local_csv_file}")


Skipping preview - data already in S3
  Run queries in Step 6 to view data


In [9]:
# Upload raw CSV to S3
s3_raw_file_path = f"{s3_raw_path}/{local_csv_file.name}"

if skip_ingestion and raw_data_exists:
    print(f"Skipping upload - data already exists in S3")
    print(f"  S3 URI: {s3_raw_file_path}")
    
    # Store the S3 file path
    %store s3_raw_file_path
    
elif not local_csv_file.exists():
    print(f"Cannot upload - local CSV file not found")
    print(f"  Expected location: {local_csv_file}")
    print(f"  Please download the file first (Step 3) or data already exists in S3")
    
else:
    print(f"Uploading data to S3...")
    print(f"  Source: {local_csv_file}")
    print(f"  Destination: {s3_raw_path}/")

    try:
        # Upload file with progress callback
        file_size = local_csv_file.stat().st_size
        
        def upload_progress(bytes_uploaded):
            percent = (bytes_uploaded / file_size) * 100
            print(f"\rUpload progress: {percent:.1f}%", end="")
        
        s3_client.upload_file(
            str(local_csv_file),
            bucket,
            f"{raw_data_prefix}/{local_csv_file.name}",
            Callback=upload_progress
        )
        
        print(f"\nUpload complete")
        print(f"  S3 URI: {s3_raw_file_path}")
        
        # Store the S3 file path
        %store s3_raw_file_path
        
    except Exception as e:
        print(f"\nError uploading to S3: {e}")

Skipping upload - data already exists in S3
  S3 URI: s3://cmsopenpaymentsystemslight/cms-open-payments-light/raw/headers.csv
Stored 's3_raw_file_path' (str)


In [10]:
# Verify upload
print("Verifying S3 upload...")

response = s3_client.list_objects_v2(
    Bucket=bucket,
    Prefix=raw_data_prefix
)

if 'Contents' in response:
    print(f"\nFiles in S3 bucket:")
    for obj in response['Contents']:
        size_gb = obj['Size'] / (1024**3)
        print(f"  {obj['Key']} ({size_gb:.2f} GB)")
else:
    print(f"\nNo files found in S3 bucket")

Verifying S3 upload...

Files in S3 bucket:
  cms-open-payments-light/raw/lightdataset.csv (0.51 GB)


## 5. Create Athena Database

Create an Amazon Athena database for querying CMS data using SQL.

In [11]:
# Set S3 staging directory for Athena queries
s3_athena_staging = f"s3://{bucket}/athena/staging"

print(f"Athena Configuration:")
print(f"  Database: {database_name}")
print(f"  Staging Directory: {s3_athena_staging}")

# Store for use in other notebooks
%store database_name
%store s3_athena_staging

Athena Configuration:
  Database: cms_open_payments_light
  Staging Directory: s3://cmsopenpaymentsystemslight/athena/staging
Stored 'database_name' (str)
Stored 's3_athena_staging' (str)


In [12]:
# Create Athena connection
athena_conn = connect(
    region_name=region,
    s3_staging_dir=s3_athena_staging
)

print("Athena connection established")

Athena connection established


In [13]:
# Create database
create_db_query = f"CREATE DATABASE IF NOT EXISTS {database_name}"

print(f"Creating Athena database...")
print(f"  Query: {create_db_query}")

try:
    result = pd.read_sql(create_db_query, athena_conn)
    print(f"Database created successfully")
except Exception as e:
    print(f"Error creating database: {e}")

Creating Athena database...
  Query: CREATE DATABASE IF NOT EXISTS cms_open_payments_light
Database created successfully


In [14]:
# Verify database creation
show_db_query = "SHOW DATABASES"

print("Verifying database creation...")
databases = pd.read_sql(show_db_query, athena_conn)

print(f"\n Available Databases:")
display(databases)

if database_name in databases.values:
    print(f"\nDatabase '{database_name}' exists")
else:
    print(f"\nDatabase '{database_name}' not found")

Verifying database creation...

 Available Databases:


,database_name
0,assignment_2
1,cms_open_payments
2,cms_open_payments_light
3,default
4,dsoaws
5,sagemaker_featurestore



Database 'cms_open_payments_light' exists


## 6. Register Data with Athena

Create an external table in Athena to query the CSV data stored in S3.

In [15]:
# Define table name
table_name_csv = "general_payments_csv"

print(f"Table Configuration:")
print(f"  Database: {database_name}")
print(f"  Table: {table_name_csv}")
print(f"  Location: {s3_raw_path}/")

%store table_name_csv

Table Configuration:
  Database: cms_open_payments_light
  Table: general_payments_csv
  Location: s3://cmsopenpaymentsystemslight/cms-open-payments-light/raw/
Stored 'table_name_csv' (str)


In [16]:
# Get actual column names from the CSV
df_schema = pd.read_csv(local_csv_file, nrows=1)

# Create column definitions for Athena
# Map pandas dtypes to Athena types
def get_athena_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'BIGINT'
    elif pd.api.types.is_float_dtype(dtype):
        return 'DOUBLE'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'TIMESTAMP'
    else:
        return 'STRING'

# Create column definitions
columns_def = []
for col in df_schema.columns:
    # Clean column name for Athena (replace spaces and special chars)
    clean_col = col.replace(' ', '_').replace('(', '').replace(')', '').replace('-', '_')
    athena_type = get_athena_type(df_schema[col].dtype)
    columns_def.append(f"`{col}` {athena_type}")

columns_str = ',\n    '.join(columns_def)

print(f"Schema preview (first 10 columns):")
for i, col_def in enumerate(columns_def[:10], 1):
    print(f"  {i}. {col_def}")
print(f"  ... ({len(columns_def)} columns total)")

Schema preview (first 10 columns):
  1. `Change_Type` STRING
  2. `Covered_Recipient_Type` STRING
  3. `Teaching_Hospital_CCN` DOUBLE
  4. `Teaching_Hospital_ID` DOUBLE
  5. `Teaching_Hospital_Name` STRING
  6. `Covered_Recipient_Profile_ID` DOUBLE
  7. `Covered_Recipient_NPI` DOUBLE
  8. `Covered_Recipient_First_Name` DOUBLE
  9. `Covered_Recipient_Middle_Name` DOUBLE
  10. `Covered_Recipient_Last_Name` DOUBLE
  ... (91 columns total)


In [17]:
# Create external table for CSV data
create_table_query = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {database_name}.{table_name_csv} (
    {columns_str}
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
LINES TERMINATED BY '\\n'
STORED AS TEXTFILE
LOCATION '{s3_raw_path}/'
TBLPROPERTIES (
    'skip.header.line.count'='1',
    'serialization.null.format'=''
)
"""

print(f"Creating external table...")
print(f"\nQuery preview:")
print(create_table_query[:500] + "...")

try:
    result = pd.read_sql(create_table_query, athena_conn)
    print(f"\nTable '{table_name_csv}' created successfully")
except Exception as e:
    print(f"\nError creating table: {e}")

Creating external table...

Query preview:

CREATE EXTERNAL TABLE IF NOT EXISTS cms_open_payments_light.general_payments_csv (
    `Change_Type` STRING,
    `Covered_Recipient_Type` STRING,
    `Teaching_Hospital_CCN` DOUBLE,
    `Teaching_Hospital_ID` DOUBLE,
    `Teaching_Hospital_Name` STRING,
    `Covered_Recipient_Profile_ID` DOUBLE,
    `Covered_Recipient_NPI` DOUBLE,
    `Covered_Recipient_First_Name` DOUBLE,
    `Covered_Recipient_Middle_Name` DOUBLE,
    `Covered_Recipient_Last_Name` DOUBLE,
    `Covered_Recipient_Name_Suffix` D...

Table 'general_payments_csv' created successfully


In [18]:
# Verify table creation
show_tables_query = f"SHOW TABLES IN {database_name}"

print("Verifying table creation...")
tables = pd.read_sql(show_tables_query, athena_conn)

print(f"\nTables in database '{database_name}':")
display(tables)

if table_name_csv in tables.values:
    print(f"\nTable '{table_name_csv}' exists")
else:
    print(f"\nTable '{table_name_csv}' not found")

Verifying table creation...

Tables in database 'cms_open_payments_light':


,tab_name
0,general_payments_csv
1,general_payments_parquet



Table 'general_payments_csv' exists


In [19]:
# Test query - count rows
count_query = f"""
SELECT COUNT(*) as row_count
FROM {database_name}.{table_name_csv}
"""

print("Testing table access...")
print(f"Query: {count_query}")

try:
    result = pd.read_sql(count_query, athena_conn)
    print(f"\nQuery successful")
    print(f"  Total rows: {result['row_count'][0]:,}")
except Exception as e:
    print(f"\nError querying table: {e}")

Testing table access...
Query: 
SELECT COUNT(*) as row_count
FROM cms_open_payments_light.general_payments_csv


Query successful
  Total rows: 1,000,000


In [20]:
# Sample query - preview data
sample_query = f"""
SELECT *
FROM {database_name}.{table_name_csv}
LIMIT 5
"""

print("Fetching sample data...")

try:
    sample_data = pd.read_sql(sample_query, athena_conn)
    print(f"\nSample data retrieved")
    print(f"  Shape: {sample_data.shape}")
    display(sample_data.head())
except Exception as e:
    print(f"\nError fetching sample data: {e}")

Fetching sample data...

Sample data retrieved
  Shape: (5, 91)


,change_type,covered_recipient_type,teaching_hospital_ccn,teaching_hospital_id,teaching_hospital_name,covered_recipient_profile_id,covered_recipient_npi,covered_recipient_first_name,covered_recipient_middle_name,covered_recipient_last_name,covered_recipient_name_suffix,recipient_primary_business_street_address_line1,recipient_primary_business_street_address_line2,recipient_city,recipient_state,recipient_zip_code,recipient_country,recipient_province,recipient_postal_code,covered_recipient_primary_type_1,covered_recipient_primary_type_2,covered_recipient_primary_type_3,covered_recipient_primary_type_4,covered_recipient_primary_type_5,covered_recipient_primary_type_6,...,indicate_drug_or_biological_or_device_or_medical_supply_2,product_category_or_therapeutic_area_2,name_of_drug_or_biological_or_device_or_medical_supply_2,associated_drug_or_biological_ndc_2,associated_device_or_medical_supply_pdi_2,covered_or_noncovered_indicator_3,indicate_drug_or_biological_or_device_or_medical_supply_3,product_category_or_therapeutic_area_3,name_of_drug_or_biological_or_device_or_medical_supply_3,associated_drug_or_biological_ndc_3,associated_device_or_medical_supply_pdi_3,covered_or_noncovered_indicator_4,indicate_drug_or_biological_or_device_or_medical_supply_4,product_category_or_therapeutic_area_4,name_of_drug_or_biological_or_device_or_medical_supply_4,associated_drug_or_biological_ndc_4,associated_device_or_medical_supply_pdi_4,covered_or_noncovered_indicator_5,indicate_drug_or_biological_or_device_or_medical_supply_5,product_category_or_therapeutic_area_5,name_of_drug_or_biological_or_device_or_medical_supply_5,associated_drug_or_biological_ndc_5,associated_device_or_medical_supply_pdi_5,program_year,payment_publication_date
0,NEW,Covered Recipient Physician,None,None,None,121431.0,1.669764e+09,None,None,None,None,4645 NW 8TH AVE,None,GAINESVILLE,FL,32605.0,United States,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2024.0,06/30/2025
1,NEW,Covered Recipient Physician,None,None,None,1177044.0,1.598078e+09,None,None,None,None,4921 PARKVIEW PL,None,"STE 5C""",SAINT LOUIS,NaN,63110,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None
2,NEW,Covered Recipient Non-Physician Practitioner,None,None,None,11193457.0,1.417324e+09,None,None,None,None,3051 BUFFALO RD,None,LAWRENCEBURG,TN,NaN,United States,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None
3,NEW,Covered Recipient Physician,None,None,None,2792925.0,1.396190e+09,None,None,None,None,12670 CREEKSIDE LN STE 202,None,FORT MYERS,FL,33919.0,United States,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2024.0,06/30/2025
4,NEW,Covered Recipient Non-Physician Practitioner,None,None,None,11108217.0,1.891773e+09,None,None,None,None,421 SE ALFRED MARKHAM ST,None,LAKE CITY,FL,NaN,United States,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,2024


## 7. Convert CSV to Parquet

Convert the CSV data to Parquet format for better performance and compression.

In [21]:
# Define Parquet table name
table_name_parquet = "general_payments_parquet"

print(f"Parquet Conversion Configuration:")
print(f"  Source Table: {database_name}.{table_name_csv}")
print(f"  Target Table: {database_name}.{table_name_parquet}")
print(f"  Target Location: {s3_parquet_path}/")

%store table_name_parquet

Parquet Conversion Configuration:
  Source Table: cms_open_payments_light.general_payments_csv
  Target Table: cms_open_payments_light.general_payments_parquet
  Target Location: s3://cmsopenpaymentsystemslight/cms-open-payments-light/parquet/
Stored 'table_name_parquet' (str)


In [30]:
# clean up the parquet files, as it was causing weird issues for me (JN)
# probably don't need to do this? 
print("Cleaning up physical S3 files...")
try:
    wr.s3.delete_objects(f"s3://{bucket}/{parquet_data_prefix}")
    print("S3 Parquet folder cleared.")
except Exception as e:
    print(f"S3 cleanup skipped or failed (might already be empty): {e}")

# removes the table definition from Glue/Athena
print("\nDropping the old table metadata...")
try:
    cursor = athena_conn.cursor()
    cursor.execute(f"DROP TABLE IF EXISTS {database_name}.{table_name_parquet}")
    print(f"Table {table_name_parquet} dropped successfully.")
except Exception as e:
    print(f"Error dropping table: {e}")


Cleaning up physical S3 files...
S3 Parquet folder cleared.

Dropping the old table metadata...
Table general_payments_parquet dropped successfully.


*** SIGTERM received at time=1770199681 on cpu 2 ***
PC: @     0x7f5f9fdcfe9e  (unknown)  epoll_wait
    @     0x7f5f47806b0d         64  absl::lts_20240722::AbslFailureSignalHandler()
    @     0x7f5f9fcec520  (unknown)  (unknown)
[2026-02-04 10:08:01,765 E 1287 1287] logging.cc:497: *** SIGTERM received at time=1770199681 on cpu 2 ***
[2026-02-04 10:08:01,765 E 1287 1287] logging.cc:497: PC: @     0x7f5f9fdcfe9e  (unknown)  epoll_wait
[2026-02-04 10:08:01,767 E 1287 1287] logging.cc:497:     @     0x7f5f47806b39         64  absl::lts_20240722::AbslFailureSignalHandler()
[2026-02-04 10:08:01,767 E 1287 1287] logging.cc:497:     @     0x7f5f9fcec520  (unknown)  (unknown)


In [24]:
# define the missing query first
columns_query = f"SELECT * FROM {database_name}.{table_name_csv} LIMIT 1"

# define columns that MUST be strings to prevent schema mismatch errors
force_string_cols = [
    'teaching_hospital_ccn', 
    'applicable_manufacturer_or_applicable_gpo_making_payment_id'
]

try:
    # get all columns from source table
    columns_df = pd.read_sql(columns_query, athena_conn)
    columns = columns_df.columns.tolist()
    
    # remove program_year if it exists (it will be added manually in the SELECT)
    if 'program_year' in columns:
        columns.remove('program_year')
    
    # create column selection string with explicit CASTing
    formatted_columns = []
    for col in columns:
        if col in force_string_cols:
            # this forces the column to be a String (VARCHAR) in the Parquet file
            formatted_columns.append(f"CAST({col} AS VARCHAR) AS {col}")
        else:
            formatted_columns.append(col)
            
    columns_str = ',\n        '.join(formatted_columns)
    
    # 3. create the query
    create_parquet_query = f"""
    CREATE TABLE {database_name}.{table_name_parquet}
    WITH (
        format = 'PARQUET',
        parquet_compression = 'SNAPPY',
        external_location = '{s3_parquet_path}/',
        partitioned_by = ARRAY['program_year']
    )
    AS
    SELECT 
        {columns_str},
        '2024' as program_year
    FROM {database_name}.{table_name_csv}
    """
    
    print("Converting CSV to Parquet format with enforced schema...")
    print(f"Query:\n{create_parquet_query}")

    # execute conversion
    pd.read_sql(create_parquet_query, athena_conn)
    print(f"\nConversion complete. Table '{table_name_parquet}' created correctly.")

except Exception as e:
    print(f"\nError during conversion: {e}")

Converting CSV to Parquet format with enforced schema...
Query:

    CREATE TABLE cms_open_payments_light.general_payments_parquet
    WITH (
        format = 'PARQUET',
        parquet_compression = 'SNAPPY',
        external_location = 's3://cmsopenpaymentsystemslight/cms-open-payments-light/parquet/',
        partitioned_by = ARRAY['program_year']
    )
    AS
    SELECT 
        change_type,
        covered_recipient_type,
        CAST(teaching_hospital_ccn AS VARCHAR) AS teaching_hospital_ccn,
        teaching_hospital_id,
        teaching_hospital_name,
        covered_recipient_profile_id,
        covered_recipient_npi,
        covered_recipient_first_name,
        covered_recipient_middle_name,
        covered_recipient_last_name,
        covered_recipient_name_suffix,
        recipient_primary_business_street_address_line1,
        recipient_primary_business_street_address_line2,
        recipient_city,
        recipient_state,
        recipient_zip_code,
        recipient_cou

In [25]:
# verify Parquet table
count_parquet_query = f"""
SELECT COUNT(*) as row_count
FROM {database_name}.{table_name_parquet}
"""

print("Verifying Parquet table...")

try:
    result = pd.read_sql(count_parquet_query, athena_conn)
    print(f"\nParquet table verified")
    print(f"  Total rows: {result['row_count'][0]:,}")
except Exception as e:
    print(f"\nError verifying Parquet table: {e}")

Verifying Parquet table...

Parquet table verified
  Total rows: 1,000,000


In [26]:
# Compare file sizes
print("Comparing CSV vs Parquet storage:")

# Get CSV size
csv_objects = s3_client.list_objects_v2(
    Bucket=bucket,
    Prefix=raw_data_prefix
)

csv_size = sum(obj['Size'] for obj in csv_objects.get('Contents', []))

# Get Parquet size
parquet_objects = s3_client.list_objects_v2(
    Bucket=bucket,
    Prefix=parquet_data_prefix
)

parquet_size = sum(obj['Size'] for obj in parquet_objects.get('Contents', []))

print(f"\nStorage Comparison:")
print(f"  CSV Size: {csv_size / (1024**3):.2f} GB")
print(f"  Parquet Size: {parquet_size / (1024**3):.2f} GB")
if parquet_size > 0:
    compression_ratio = (1 - parquet_size/csv_size) * 100
    print(f"  Compression: {compression_ratio:.1f}% reduction")
    print(f"  Space Saved: {(csv_size - parquet_size) / (1024**3):.2f} GB")

Comparing CSV vs Parquet storage:

Storage Comparison:
  CSV Size: 0.51 GB
  Parquet Size: 0.06 GB
  Compression: 89.0% reduction
  Space Saved: 0.46 GB


## 8. Query Data with AWS Data Wrangler

Use AWS Data Wrangler for more efficient data querying and analysis.

In [27]:
# Query using AWS Data Wrangler
sample_query_wr = f"""
SELECT 
    COUNT(*) as total_payments,
    SUM(CAST(Total_Amount_of_Payment_USDollars AS DOUBLE)) as total_amount,
    AVG(CAST(Total_Amount_of_Payment_USDollars AS DOUBLE)) as avg_amount,
    MIN(CAST(Total_Amount_of_Payment_USDollars AS DOUBLE)) as min_amount,
    MAX(CAST(Total_Amount_of_Payment_USDollars AS DOUBLE)) as max_amount
FROM {database_name}.{table_name_parquet}
"""

print("Querying payment statistics with AWS Data Wrangler...")
print(f"\nQuery: {sample_query_wr}")

try:
    df_stats = wr.athena.read_sql_query(
        sql=sample_query_wr,
        database=database_name,
        ctas_approach=False
    )
    
    print(f"\nQuery successful")
    print(f"\nPayment Statistics:")
    display(df_stats)
    
except Exception as e:
    print(f"\nError querying data: {e}")

Querying payment statistics with AWS Data Wrangler...

Query: 
SELECT 
    COUNT(*) as total_payments,
    SUM(CAST(Total_Amount_of_Payment_USDollars AS DOUBLE)) as total_amount,
    AVG(CAST(Total_Amount_of_Payment_USDollars AS DOUBLE)) as avg_amount,
    MIN(CAST(Total_Amount_of_Payment_USDollars AS DOUBLE)) as min_amount,
    MAX(CAST(Total_Amount_of_Payment_USDollars AS DOUBLE)) as max_amount
FROM cms_open_payments_light.general_payments_parquet


Query successful

Payment Statistics:


,total_payments,total_amount,avg_amount,min_amount,max_amount
0,1000000,4.500100e+12,8.206366e+06,0.01,1.000010e+11


In [28]:
# Sample data by recipient type
recipient_query = f"""
SELECT 
    Covered_Recipient_Type,
    COUNT(*) as payment_count,
    SUM(CAST(Total_Amount_of_Payment_USDollars AS DOUBLE)) as total_amount
FROM {database_name}.{table_name_parquet}
GROUP BY Covered_Recipient_Type
ORDER BY total_amount DESC
"""

print("Analyzing payments by recipient type...")

try:
    df_recipients = wr.athena.read_sql_query(
        sql=recipient_query,
        database=database_name,
        ctas_approach=False
    )
    
    print(f"\nQuery successful")
    print(f"\nPayments by Recipient Type:")
    display(df_recipients)
    
except Exception as e:
    print(f"\nError querying data: {e}")

Analyzing payments by recipient type...

Query successful

Payments by Recipient Type:


,Covered_Recipient_Type,payment_count,total_amount
0,Covered Recipient Physician,642145,2.800069e+12
1,Covered Recipient Non-Physician Practitioner,355575,1.700011e+12
2,Covered Recipient Teaching Hospital,2280,1.961271e+07


## 9. Validation & Verification

Perform final validation checks on the datalake setup.

In [29]:
# Comprehensive validation
print("=" * 70)
print("DATALAKE SETUP VALIDATION")
print("=" * 70)

validation_passed = True

# Check 1: S3 Buckets
print("\n1. S3 Storage:")
try:
    for prefix in [raw_data_prefix, parquet_data_prefix]:
        response = s3_client.list_objects_v2(Bucket=bucket, Prefix=prefix, MaxKeys=1)
        if 'Contents' in response:
            print(f"  {prefix}/")
        else:
            print(f"  [FAIL]{prefix}/ (empty or missing)")
            validation_passed = False
except Exception as e:
    print(f"Error checking S3: {e}")
    validation_passed = False

# Check 2: Athena Database
print("\n2. Athena Database:")
try:
    databases = pd.read_sql("SHOW DATABASES", athena_conn)
    if database_name in databases.values:
        print(f"Database '{database_name}' exists")
    else:
        print(f"Database '{database_name}' not found")
        validation_passed = False
except Exception as e:
    print(f"Error checking database: {e}")
    validation_passed = False

# Check 3: Tables
print("\n3. Athena Tables:")
try:
    tables = pd.read_sql(f"SHOW TABLES IN {database_name}", athena_conn)
    for table in [table_name_csv, table_name_parquet]:
        if table in tables.values:
            print(f"Table '{table}' exists")
        else:
            print(f"Table '{table}' not found")
            validation_passed = False
except Exception as e:
    print(f"Error checking tables: {e}")
    validation_passed = False

# Check 4: Data Accessibility
print("\n4. Data Accessibility:")
try:
    count_result = pd.read_sql(
        f"SELECT COUNT(*) as cnt FROM {database_name}.{table_name_parquet}",
        athena_conn
    )
    row_count = count_result['cnt'][0]
    print(f"Query successful ({row_count:,} rows)")
except Exception as e:
    print(f"Error querying data: {e}")
    validation_passed = False

# Final result
print("\n" + "=" * 70)
if validation_passed:
    print("ALL VALIDATION CHECKS PASSED")
    print("Datalake setup complete and operational")
    setup_datalake_passed = True
else:
    print("SOME VALIDATION CHECKS FAILED")
    print("Please review the errors above and re-run failed steps")
    setup_datalake_passed = False

print("=" * 70)

# Store validation result
%store setup_datalake_passed

DATALAKE SETUP VALIDATION

1. S3 Storage:
  cms-open-payments-light/raw/
  cms-open-payments-light/parquet/

2. Athena Database:
Database 'cms_open_payments_light' exists

3. Athena Tables:
Table 'general_payments_csv' exists
Table 'general_payments_parquet' exists

4. Data Accessibility:
Query successful (1,000,000 rows)

ALL VALIDATION CHECKS PASSED
Datalake setup complete and operational
Stored 'setup_datalake_passed' (bool)
